## File to implement nn with wandb
(note that we can't run wandb in terminal)

In [1]:
# file to load the nn

import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import wandb
from wandb.keras import WandbCallback

# read in our data
DATA_DIR = '/home/oscar47/Desktop/astro101/data/g_band/var_output/v0.1.1'

# check if keras recognizes gpu
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

train_x_ds = np.load(os.path.join(DATA_DIR, 'train_x_ds.npy'))
val_x_ds = np.load(os.path.join(DATA_DIR, 'val_x_ds.npy'))
train_y_ds = np.load(os.path.join(DATA_DIR, 'train_y_ds.npy'))
val_y_ds = np.load(os.path.join(DATA_DIR, 'val_y_ds.npy'))

input_shape = train_x_ds[0].shape
output_len = len(train_y_ds[0])

# build model functions--------------------------------
def build_model(size1, size2, size3, size4, size5, dropout, learning_rate):
    model = Sequential()

    model.add(layers.Dense(size1))
    model.add(layers.Dense(size2))
    model.add(layers.Dense(size3))
    model.add(layers.Dense(size4))
    model.add(layers.Dense(size5))

    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(output_len))

    # return len of class size
    model.add(layers.Dense(output_len))
    model.add(layers.Activation('softmax'))

    optimizer = Adam(learning_rate = learning_rate, clipnorm=1)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')

    return model


# build model functions--------------------------------
def build_model_mini(size1, learning_rate):
    model = Sequential()

    model.add(layers.Dense(size1))

    # return len of class size
    model.add(layers.Dense(output_len))
    model.add(layers.Activation('softmax'))

    optimizer = Adam(learning_rate = learning_rate, clipnorm=1)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')

    return model

# build model functions--------------------------------
def build_model_test(size1, dropout):
    model = Sequential()

    model.add(layers.Dense(size1))

    # return len of class size
    model.add(layers.Dense(output_len))
    model.add(layers.Activation('softmax'))

    optimizer = Adam()
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')

    return model


def train(config=None):
    with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
      config = wandb.config

      #pprint.pprint(config)

      #initialize the neural net; 
      global model
      model = build_model(config.size_1,  config.size_2, config.size_3, 
              config.size_4, config.size_5, 
              config.dropout, config.learning_rate)
      
      #now run training
      history = model.fit(
        train_x_ds, train_y_ds,
        batch_size = config.batch_size,
        validation_data=(val_x_ds, val_y_ds),
        epochs=config.epochs,
        callbacks=[WandbCallback()] #use callbacks to have w&b log stats; will automatically save best model                     
      )

   
def train_mini(config=None):
    with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
      config = wandb.config

      #pprint.pprint(config)

      #initialize the neural net; 
      global model
      model = build_model_mini(config.size_1, config.learning_rate)
      
      #now run training
      history = model.fit(
        train_x_ds, train_y_ds,
        batch_size = config.batch_size,
        validation_data=(val_x_ds, val_y_ds),
        epochs=config.epochs,
        callbacks=[WandbCallback()] #use callbacks to have w&b log stats; will automatically save best model                     
      )


def train_manual():
    global model
    model = build_model_mini(256, 0.2)
    
    #now run training
    history = model.fit(
    train_x_ds, train_y_ds,
    batch_size = 128,
    validation_data=(val_x_ds, val_y_ds),
    epochs=10
    )

# set dictionary with random search; optimizing val_loss--------------------------
sweep_config= {
    'method': 'random',
    'name': 'val_accuracy',
    'goal': 'maximize'
}

sweep_config['metric']= 'val_accuracy'

parameters_dict = {
    'epochs': {
       'distribution': 'int_uniform',
       'min': 20,
       'max': 100
    },
    # for build_dataset
     'batch_size': {
       'values': [x for x in range(32, 161, 32)]
    },
    'size_1': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
    'learning_rate':{
         #uniform distribution between 0 and 1
         'distribution': 'uniform', 
         'min': 0,
         'max': 0.1
     }
}


# parameters_dict = {
#     'epochs': {
#        'distribution': 'int_uniform',
#        'min': 20,
#        'max': 100
#     },
#     # for build_dataset
#      'batch_size': {
#        'values': [x for x in range(32, 161, 32)]
#     },
#     'size_1': {
#        'distribution': 'int_uniform',
#        'min': 64,
#        'max': 256
#     },
#     'size_2': {
#        'distribution': 'int_uniform',
#        'min': 64,
#        'max': 256
#     },'size_3': {
#        'distribution': 'int_uniform',
#        'min': 64,
#        'max': 256
#     },'size_4': {
#        'distribution': 'int_uniform',
#        'min': 64,
#        'max': 256
#     },'size_5': {
#        'distribution': 'int_uniform',
#        'min': 64,
#        'max': 256
#     },
#     'dropout': {
#       'distribution': 'uniform',
#        'min': 0,
#        'max': 0.6
#     },
#     'learning_rate':{
#          #uniform distribution between 0 and 1
#          'distribution': 'uniform', 
#          'min': 0,
#          'max': 0.1
#      }
# }

# append parameters to sweep config
sweep_config['parameters'] = parameters_dict 

# login to wandb----------------
wandb.init(project="Astro101_Project_Mini", entity="oscarscholin")

# initialize sweep agent
sweep_id = wandb.sweep(sweep_config, project='Astro101_Project_Mini', entity="oscarscholin")
wandb.agent(sweep_id, train_mini, count=100)

#train_manual()

2022-12-11 19:51:20.277881: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-11 19:51:20.396694: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-11 19:51:20.398987: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-11 19:51:20.398996: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc

Num GPUs Available:  0


2022-12-11 19:51:21.437166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 19:51:21.437640: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-11 19:51:21.437661: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-11 19:51:21.437677: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-11 19:51:21.437694: W tensorflow/c

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('goal' was unexpected)
wandb: WARNING   Violation 2. 'val_accuracy' is not of type 'object'
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: pbxmtjrk
Sweep URL: https://wandb.ai/oscarscholin/Astro101_Project_Mini/sweeps/pbxmtjrk


...

1184/1184 [==============================] - 1s 891us/step - loss: 0.5555 - val_loss: 0.5168
Epoch 12/47
1184/1184 [==============================] - 1s 699us/step - loss: 0.5726 - val_loss: 0.5749
Epoch 13/47
1184/1184 [==============================] - 1s 645us/step - loss: 0.5544 - val_loss: 0.5570
Epoch 14/47
1184/1184 [==============================] - 1s 666us/step - loss: 0.5540 - val_loss: 0.5482
Epoch 15/47
1184/1184 [==============================] - 1s 655us/step - loss: 0.5675 - val_loss: 0.5651
Epoch 16/47
1184/1184 [==============================] - 1s 668us/step - loss: 0.5533 - val_loss: 0.5930
Epoch 17/47
1184/1184 [==============================] - 1s 730us/step - loss: 0.5644 - val_loss: 0.5373
Epoch 18/47
1184/1184 [==============================] - 1s 688us/step - loss: 0.5642 - val_loss: 0.5659
Epoch 19/47
1184/1184 [==============================] - 1s 657us/step - loss: 0.5493 - val_loss: 0.5808
Epoch 20/47
1184/1184 [==============================] - 1s 695us/s

wandb: Adding directory to artifact (/home/oscar47/Desktop/astro101/Astro101-Final-Project/nn_v0.0.1/wandb/run-20221211_223553-b6kdi94a/files/model-best)... Done. 0.0s


1184/1184 [==============================] - 1s 890us/step - loss: 0.5454 - val_loss: 0.5077
Epoch 22/47
1184/1184 [==============================] - 1s 720us/step - loss: 0.5535 - val_loss: 0.5307
Epoch 23/47
1184/1184 [==============================] - 1s 674us/step - loss: 0.5508 - val_loss: 0.5440
Epoch 24/47
1184/1184 [==============================] - 1s 742us/step - loss: 0.5528 - val_loss: 0.5274
Epoch 25/47
1184/1184 [==============================] - 1s 663us/step - loss: 0.5586 - val_loss: 0.5295
Epoch 26/47
1184/1184 [==============================] - 1s 713us/step - loss: 0.5498 - val_loss: 0.5192
Epoch 27/47
1184/1184 [==============================] - 1s 742us/step - loss: 0.5455 - val_loss: 0.5594
Epoch 28/47
1184/1184 [==============================] - 1s 707us/step - loss: 0.5515 - val_loss: 0.5267
Epoch 29/47
1184/1184 [==============================] - 1s 686us/step - loss: 0.5523 - val_loss: 0.5231
Epoch 30/47
1184/1184 [==============================] - 1s 650us/s

wandb: Adding directory to artifact (/home/oscar47/Desktop/astro101/Astro101-Final-Project/nn_v0.0.1/wandb/run-20221211_223553-b6kdi94a/files/model-best)... Done. 0.0s


1184/1184 [==============================] - 1s 954us/step - loss: 0.5511 - val_loss: 0.4948
Epoch 36/47
1184/1184 [==============================] - 1s 676us/step - loss: 0.5439 - val_loss: 0.5265
Epoch 37/47
1184/1184 [==============================] - 1s 576us/step - loss: 0.5446 - val_loss: 0.5222
Epoch 38/47
1184/1184 [==============================] - 1s 729us/step - loss: 0.5534 - val_loss: 0.5655
Epoch 39/47
1184/1184 [==============================] - 1s 705us/step - loss: 0.5530 - val_loss: 0.5645
Epoch 40/47
1184/1184 [==============================] - 1s 705us/step - loss: 0.5464 - val_loss: 0.5531
Epoch 41/47
1184/1184 [==============================] - 1s 614us/step - loss: 0.5392 - val_loss: 0.5569
Epoch 42/47
1184/1184 [==============================] - ETA: 0s - loss: 0.5461